<a href="https://colab.research.google.com/github/EmTampz/Copra_YOLOv11_v12_Colab_Notebooks/blob/main/Copra_1_manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# 📦 ENVIRONMENT SETUP for YOLOv11 Thesis (Colab)
# ==========================================================
# This installs only what is required for:
# - YOLOv11 object detection, classification, segmentation
# - Grid search tuning and visualization
# - Logging and performance tracking
# ==========================================================

# Core deep learning and YOLO library
!pip install -U ultralytics

# Core data handling, visualization, and ML utilities
!pip install pandas matplotlib seaborn pillow scikit-learn

# Optional (but recommended) for monitoring and logging
!pip install tensorboard tqdm

# Example Roboflow integration
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
import torch, ultralytics, pandas as pd, matplotlib, seaborn, PIL, sklearn, tqdm, tensorboard

print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available  : {torch.cuda.is_available()}")
print(f"Ultralytics ver : {ultralytics.__version__}")
ultralytics.checks()

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.3/112.6 GB disk)


In [ ]:
"""
manual_tuning_yolo11.py
Manual hyperparameter tuning for YOLOv11 (Detection -> Segmentation -> Classification)
Integrated with Roboflow API for automatic dataset download (YOLO format).
Runs on Google Colab GPU with TensorBoard logging, timestamps, CSV results,
limited inference visualization for quick inspection, and side-by-side comparison.
Automatically zips all inference results for download.

Before running:
!pip install ultralytics pandas matplotlib seaborn pillow roboflow
"""

import os, time, shutil, yaml, json, random, glob, zipfile
from datetime import datetime
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import Image as IPyImage, display   # ✅ for inline visualization in Colab

# ---------------- USER CONFIG ----------------
# ✅ Roboflow integration — replace with your API key and dataset details
rf = Roboflow(api_key="LS1Ohi2OQZ178OOqQ4zc")
project = rf.workspace("cv-opfhv").project("mor-v4-8itha")
version = project.version(1)
dataset = version.download("yolov11")

# Dynamically detect the dataset YAML path
ROBOFLOW_DATA_YAML = os.path.join(dataset.location, "data.yaml")
print(f"✅ Dataset successfully downloaded from Roboflow!\n📂 YAML path: {ROBOFLOW_DATA_YAML}")

RESULTS_BASE = "/content/copra_yolo11_manual_results"
os.makedirs(RESULTS_BASE, exist_ok=True)

# # Tasks: Detection → Segmentation → Classification
TASKS = ["detect", "seg", "cls"]
MODEL_MAP = {
    "detect": "yolo11n.pt",
    "seg":    "yolo11n-seg.pt",
    "cls":    "yolo11n-cls.pt"
}

# # Manual hyperparameters
MANUAL_CONFIG = {
    "epochs": 100,
    "batch": 16,
    "optimizer": "auto",
    "lr0": 0.001,
    "patience": 100
}

IMG_SIZE = 640
TENSORBOARD_ENABLE = True

# CSV to append results
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
SUMMARY_CSV = os.path.join(RESULTS_BASE, f"manual_summary_{TIMESTAMP}.csv")

# ---------------- Helper utilities ----------------
def load_data_yaml(path):
    with open(path, "r") as f:
        return yaml.safe_load(f)

# Compute IoU between predicted and actual bounding boxes used for Evaluating detection accuracy, computing mAP
def iou_xyxy(box1, box2):
    x1 = max(box1[0], box2[0]); y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2]); y2 = min(box1[3], box2[3])
    inter_w = max(0, x2 - x1); inter_h = max(0, y2 - y1)
    inter_area = inter_w * inter_h
    area1 = max(0, (box1[2] - box1[0]) * (box1[3] - box1[1]))
    area2 = max(0, (box2[2] - box2[0]) * (box2[3] - box2[1]))
    union = area1 + area2 - inter_area
    return inter_area / union if union > 0 else 0.0

# Convert normalized YOLO coordinates to pixel coordinates, Preparing data for IoU computation, visualization
def yolo_label_to_xyxy(norm, w, h):
    xc, yc, bw, bh = norm
    xc *= w; yc *= h; bw *= w; bh *= h
    x1 = xc - bw/2; y1 = yc - bh/2; x2 = xc + bw/2; y2 = yc + bh/2
    return [x1, y1, x2, y2]

# Reads YOLO label files and returns structured annotations, Dataset quality checks, IoU computation
def parse_label_file(label_path):
    labels = []
    if not os.path.exists(label_path):
        return labels
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls = int(parts[0]); norm = list(map(float, parts[1:5]))
                labels.append((cls, norm))
    return labels

# Automatically enables TensorBoard for Live training visualization
def ensure_tensorboard():
    if TENSORBOARD_ENABLE:
        os.system("yolo settings tensorboard=True")

# Copies trained .pt model with run details
def save_weights_copy(src_path, dest_dir, run_name):
    os.makedirs(dest_dir, exist_ok=True)
    if src_path and os.path.exists(src_path):
        dest = os.path.join(dest_dir, f"{run_name}_{Path(src_path).name}")
        shutil.copy(src_path, dest)
        return dest
    return None

# ---------------- Main per-task run ----------------
def run_one_task(task, base_model, cfg, results_base):
    assert task in MODEL_MAP
    model = YOLO(base_model)
    run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"{task}_manual_e{cfg['epochs']}_b{cfg['batch']}_p{cfg['patience']}_opt{cfg['optimizer']}_lr{cfg['lr0']}_{run_timestamp}"
    project_dir = os.path.join(results_base, task, run_name)
    os.makedirs(project_dir, exist_ok=True)

    print(f"\n--- Training {task} | run: {run_name} ---")
    t0 = time.time()
    # Start training
    model.train(
        data=ROBOFLOW_DATA_YAML,
        epochs=cfg["epochs"],
        batch=cfg["batch"],
        imgsz=IMG_SIZE,
        optimizer=cfg["optimizer"],
        lr0=cfg["lr0"],
        patience=cfg["patience"],
        project=os.path.join(results_base, task),
        name=run_name,
        resume=False
    )
    t1 = time.time()
    elapsed = round(t1 - t0, 2)
    print(f"⏱️ Training finished in {elapsed} seconds")

    # Find best or last weight produced by Ultralytics
    weights_dir = os.path.join(results_base, task, run_name, "weights")
    best_w = os.path.join(weights_dir, "best.pt")
    last_w = os.path.join(weights_dir, "last.pt")
    chosen = best_w if os.path.exists(best_w) else (last_w if os.path.exists(last_w) else None)
    saved_copy = save_weights_copy(chosen, os.path.join(results_base, "saved_models"), run_name)

    # Validate and extract metrics
    val_results = model.val()
    metrics = {}
    try:
        metrics["mAP@0.5"] = float(val_results.box.map50)
        metrics["mAP@0.5:0.95"] = float(val_results.box.map)
        metrics["precision"] = float(val_results.box.precision)
        metrics["recall"] = float(val_results.box.recall)
        metrics["f1"] = float(val_results.box.f1)
    except Exception:
        metrics = {"mAP@0.5": None, "mAP@0.5:0.95": None, "precision": None, "recall": None, "f1": None}

    # Quick inference for visualization (limited to 3 images)
    infer_folder = os.path.join(project_dir, "inference_results")
    os.makedirs(infer_folder, exist_ok=True)
    val_images_path = os.path.join(dataset.location, "valid", "images")
    sample_imgs = random.sample(glob.glob(f"{val_images_path}/*.jpg"), min(3, len(glob.glob(f"{val_images_path}/*.jpg"))))

    if sample_imgs:
        print(f"🖼️ Running quick inference on {len(sample_imgs)} sample images...")
        model.predict(source=sample_imgs, save=True, project=infer_folder, name="sample_preds")
        pred_folder = os.path.join(infer_folder, "sample_preds")
        for img in glob.glob(f"{pred_folder}/*.jpg")[:3]:
            display(IPyImage(filename=img, width=600))  # Display few predictions only
            print(f"Displayed: {os.path.basename(img)}")

    # ---------------- Per-class metrics and plots ----------------
    data_meta = load_data_yaml(ROBOFLOW_DATA_YAML)
    class_names = data_meta.get("names", [])
    y_true, y_pred = [], []

    # Prepare per-class metrics on validation set
    val_images_dir = os.path.join(dataset.location, "valid", "images")
    for img_path in glob.glob(f"{val_images_dir}/*.jpg"):
        # YOLO prediction
        res_list = model.predict(img_path, imgsz=IMG_SIZE, conf=0.25, iou=0.45, stream=False)
        if len(res_list) == 0: continue
        res = res_list[0]
        preds = []
        if hasattr(res, "boxes") and res.boxes.data is not None:
            for b in res.boxes.data.tolist():
                x1, y1, x2, y2, score, cls = b
                preds.append({"box":[x1, y1, x2, y2], "cls": int(cls)})

        # Ground truth
        label_path = Path(img_path).with_suffix(".txt")
        if not label_path.exists():
            alt = Path(val_images_dir).parent / "labels" / label_path.name
            if alt.exists(): label_path = alt
        gt = parse_label_file(str(label_path))
        gt_bboxes = []
        try:
            w,h = Image.open(img_path).size
        except:
            w,h = IMG_SIZE, IMG_SIZE
        for cid,norm in gt:
            bb = yolo_label_to_xyxy(norm, w, h)
            gt_bboxes.append({"box": bb, "cls": cid})

        # Match preds to GT
        used = set()
        for p in preds:
            best_iou = 0; best_idx = -1
            for i, g in enumerate(gt_bboxes):
                if i in used: continue
                iou = iou_xyxy(p["box"], g["box"])
                if iou > best_iou:
                    best_iou = iou; best_idx = i
            if best_iou >= 0.5 and best_idx >= 0:
                used.add(best_idx)
                y_true.append(gt_bboxes[best_idx]["cls"])
                y_pred.append(p["cls"])
            else:
                y_true.append(-1)
                y_pred.append(p["cls"])
        # Add unmatched GTs
        for i, g in enumerate(gt_bboxes):
            if i not in used:
                y_true.append(g["cls"])
                y_pred.append(-1)

    # Compute per-class metrics
    if len(y_true) > 0:
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, labels=list(range(len(class_names))), zero_division=0
        )
        per_class_df = pd.DataFrame({
            "class": class_names,
            "precision": precision,
            "recall": recall,
            "f1": f1
        })
        per_class_csv = os.path.join(project_dir, "per_class_metrics.csv")
        per_class_df.to_csv(per_class_csv, index=False)
        # Plot per-class metrics
        fig, ax = plt.subplots(figsize=(10,5))
        x = np.arange(len(class_names))
        ax.bar(x - 0.2, precision, width=0.2, label="Precision")
        ax.bar(x, recall, width=0.2, label="Recall")
        ax.bar(x + 0.2, f1, width=0.2, label="F1-score")
        ax.set_xticks(x); ax.set_xticklabels(class_names, rotation=45, ha="right")
        ax.set_ylabel("Score")
        ax.set_title(f"Per-class Precision, Recall, F1 for {run_name}")
        ax.legend()
        plt.tight_layout()
        per_class_png = os.path.join(project_dir, "per_class_metrics.png")
        plt.savefig(per_class_png)
        plt.close()

    # Save record
    run_record = {
        "timestamp": run_timestamp,
        "task": task,
        "run_name": run_name,
        "model_source": base_model,
        "saved_model": saved_copy,
        "epochs": cfg["epochs"],
        "batch": cfg["batch"],
        "optimizer": cfg["optimizer"],
        "lr0": cfg["lr0"],
        "patience": cfg["patience"],
        "elapsed_seconds": elapsed,
        "mAP@0.5": metrics.get("mAP@0.5"),
        "mAP@0.5:0.95": metrics.get("mAP@0.5:0.95"),
        "precision": metrics.get("precision"),
        "recall": metrics.get("recall"),
        "f1": metrics.get("f1"),
        "inference_folder": infer_folder,
        "per_class_metrics_csv": per_class_csv,
        "per_class_metrics_png": per_class_png
    }
    return run_record

# ---------------- Zip everything ----------------
def zip_all_results(results_base, timestamp):
    zip_name = os.path.join(results_base, f"all_results_{timestamp}.zip")
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(results_base):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=results_base)
                zipf.write(file_path, arcname)
    print(f"\n📦 All results (weights, CSVs, plots, inference) zipped: {zip_name}")
    return zip_name

# ---------------- Entry point ----------------
if __name__ == "__main__":
    ensure_tensorboard()
    data_meta = load_data_yaml(ROBOFLOW_DATA_YAML)
    print("📊 Classes:", data_meta.get("names"))
    summary_rows = []

    for task in TASKS:
        base_model = MODEL_MAP[task]
        rec = run_one_task(task, base_model, MANUAL_CONFIG, RESULTS_BASE)
        print("Run summary:", rec)
        summary_rows.append(rec)

    # Save summary CSV
    df = pd.DataFrame(summary_rows)
    df.to_csv(SUMMARY_CSV, index=False)
    print(f"\n✅ Manual tuning complete. Summary saved to: {SUMMARY_CSV}")
    print("📂 Saved models and TensorBoard logs are in:", RESULTS_BASE)

    # Zip everything
    zip_all_results(RESULTS_BASE, TIMESTAMP)
